In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d07ef77f",
   "metadata": {},
   "outputs": [],
   "source": [
    "!wget https://raw.githubusercontent.com/spmallick/learnopencv/refs/heads/master/HandPose/hand/pose_deploy.prototxt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "52c4a4d9",
   "metadata": {},
   "outputs": [],
   "source": [
    "!wget https://huggingface.co/camenduru/openpose/resolve/5e17f6ad43ab415a0114537541a8d37d2503424f/models/hand/pose_iter_102000.caffemodel"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9d59e990",
   "metadata": {},
   "outputs": [],
   "source": [
    "!wget https://raw.githubusercontent.com/spmallick/learnopencv/refs/heads/master/HandPose/right-frontal.jpg"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "46d44580",
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "import time\n",
    "import numpy as np\n",
    "\n",
    "protoFile = \"hand/pose_deploy.prototxt\"\n",
    "weightsFile = \"hand/pose_iter_102000.caffemodel\"\n",
    "nPoints = 22\n",
    "POSE_PAIRS = [\n",
    "    [0, 1],\n",
    "    [1, 2],\n",
    "    [2, 3],\n",
    "    [3, 4],\n",
    "    [0, 5],\n",
    "    [5, 6],\n",
    "    [6, 7],\n",
    "    [7, 8],\n",
    "    [0, 9],\n",
    "    [9, 10],\n",
    "    [10, 11],\n",
    "    [11, 12],\n",
    "    [0, 13],\n",
    "    [13, 14],\n",
    "    [14, 15],\n",
    "    [15, 16],\n",
    "    [0, 17],\n",
    "    [17, 18],\n",
    "    [18, 19],\n",
    "    [19, 20],\n",
    "]\n",
    "net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2fd20e20",
   "metadata": {},
   "outputs": [],
   "source": [
    "frame = cv2.imread(\"right-frontal.jpg\")\n",
    "frameCopy = np.copy(frame)\n",
    "frameWidth = frame.shape[1]\n",
    "frameHeight = frame.shape[0]\n",
    "aspect_ratio = frameWidth / frameHeight\n",
    "threshold = 0.1\n",
    "\n",
    "t = time.time()\n",
    "# input image dimensions for the network\n",
    "inHeight = 368\n",
    "# inWidth = 368\n",
    "inWidth = int(((aspect_ratio * inHeight) * 8) // 8)\n",
    "inpBlob = cv2.dnn.blobFromImage(\n",
    "    frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False\n",
    ")\n",
    "\n",
    "net.setInput(inpBlob)\n",
    "\n",
    "output = net.forward()\n",
    "print(\"time taken by network : {:.3f}\".format(time.time() - t))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "01e6712e",
   "metadata": {},
   "outputs": [],
   "source": [
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Empty list to store the detected keypoints\n",
    "points = []\n",
    "\n",
    "for i in range(nPoints):\n",
    "    # confidence map of corresponding body's part.\n",
    "    probMap = output[0, i, :, :]\n",
    "    probMap = cv2.resize(probMap, (frameWidth, frameHeight))\n",
    "\n",
    "    # Find global maxima of the probMap.\n",
    "    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)\n",
    "\n",
    "    if prob > threshold:\n",
    "        cv2.circle(\n",
    "            frameCopy,\n",
    "            (int(point[0]), int(point[1])),\n",
    "            3,\n",
    "            (0, 255, 255),\n",
    "            thickness=-1,\n",
    "            lineType=cv2.FILLED,\n",
    "        )\n",
    "        cv2.putText(\n",
    "            frameCopy,\n",
    "            \"{}\".format(i),\n",
    "            (int(point[0]), int(point[1])),\n",
    "            cv2.FONT_HERSHEY_SIMPLEX,\n",
    "            0.8,\n",
    "            (0, 0, 255),\n",
    "            2,\n",
    "            lineType=cv2.LINE_AA,\n",
    "        )\n",
    "\n",
    "        # Add the point to the list if the probability is greater than the threshold\n",
    "        points.append((int(point[0]), int(point[1])))\n",
    "    else:\n",
    "        points.append(None)\n",
    "\n",
    "# Draw Skeleton\n",
    "for pair in POSE_PAIRS:\n",
    "    partA = pair[0]\n",
    "    partB = pair[1]\n",
    "\n",
    "    if points[partA] and points[partB]:\n",
    "        cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)\n",
    "        cv2.circle(\n",
    "            frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED\n",
    "        )\n",
    "        cv2.circle(\n",
    "            frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED\n",
    "        )\n",
    "\n",
    "\n",
    "plt.figure(figsize=[10, 10])\n",
    "plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))\n",
    "plt.figure(figsize=[10, 10])\n",
    "plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "\n",
    "\n",
    "cv2.imwrite(\"Output-Keypoints.jpg\", frameCopy)\n",
    "cv2.imwrite(\"Output-Skeleton.jpg\", frame)\n",
    "\n",
    "print(\"Total time taken : {:.3f}\".format(time.time() - t))"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}